<a href="https://colab.research.google.com/github/yang-su2000/Authorship-Identification-with-NLP/blob/main/1-conv1d_cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1D Convolution on CPU

## 1. Set-up 

In [ ]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username    
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [ ]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/ece5545’: File exists
/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-yang-su2000' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-yang-su2000
M	1-conv1d_cpu.ipynb
M	2-conv1d_gpu.ipynb
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 7 (delta 5), reused 7 (delta 5), pack-reused 0
Unpacking objects: 100% (7/7), 1.45 KiB | 0 bytes/s, done.
From https://github.com/ML-HW-SYS/a3-yang-su2000
   172b133..8c34443  main       -> origin/main
Updating 172b133..8c34443
Fast-forward
 src/ops.py               | 125 +++++++++++++++++++++++++++++++----------------
 tests/test_1dconv_cpu.py |   4 +-
 2 files changed, 84 insertions(+), 45 deletions(-)
/content/gdrive/MyDrive/ece5545


In [ ]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

Verify the following cell prints your github repository.

In [ ]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb   4-gemm_gpu.ipynb	    README.md	      tests
2-conv1d_gpu.ipynb   5-conv2d_dw_gpu.ipynb  repo_profiler.py
3-conv1d_fpga.ipynb  leaderboard_id.txt     src


## 2. Install TVM

In [ ]:
!pip --version

pip 23.0.1 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)


In [ ]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 MB 3.4 MB/s eta 0:00:00


## 3. Implement `make_conv1d_cpu_scheduler_func` function in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then 
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_cpu_scheduler_func` takes $m$ and $n$, which are the size of the two 1D input array. 
You should return both the TVM schedule and the TVM operator for 
1. Input $x$
2. Input $y$
3. Output $out$

The schedule should be able to used to build a function with signature $func(x, y, out)$. 
Please see the following cells the usage.

In [ ]:
from IPython.core.formatters import BaseFormatter
import os
import tvm
from tvm import te

def make_conv1d_cpu_scheduler1(M, N, bn=32, kfactor=32):
    # baseline
    A = te.placeholder((M,), name="A")
    W = te.placeholder((N,), name="W")
    k = te.reduce_axis((0, M + N - 1), "k")
    B = te.compute(
        (M + N - 1,),
        lambda n: te.sum(tvm.tir.if_then_else(
            tvm.tir.any(k < 0, k >= M, n - k < 0, n - k >= N),
            tvm.tir.const(0.0, "float32"),
            A[k] * W[n - k]), axis=k),
        name="B",
    )
    s = te.create_schedule(B.op)

    # Allocate write cache
    BB = s.cache_write(B, "global")

    # Blocking by loop tiling
    xo, xi = s[B].split(s[B].op.axis[0], factor=bn)

    # Write cache is computed at xo
    s[BB].compute_at(s[B], xi)

    # New inner axes
    (xc,) = s[BB].op.axis

    # Blocking by loop tiling for reduction axis
    (k,) = s[BB].op.reduce_axis
    ko, ki = s[BB].split(k, factor=kfactor)

    # Vectorization
    # s[BB].reorder(ko, xc, ki)
    # s[BB].unroll(ki)
    # s[BB].vectorize(xc)

    # Parallel
    s[BB].parallel(xc)

    # Parallel
    s[B].parallel(xo)
    
    return s, A, W, B

In [ ]:
import time
import timeit
import numpy as np

def make_conv1d_cpu_func(M, N):
    s, A, W, O = make_conv1d_cpu_scheduler1(M, N)
    func = tvm.build(s, [A, W, O], "llvm")
    return func

def test_speed_conv1d_cpu():
    # Define dimension
    dev = tvm.device('llvm', 0)
    M = 1024
    N = 1024
    n_repeat = 100
    func = make_conv1d_cpu_func(M, N)

    def tvm_time():
        # Create random test data
        np.random.seed(seed=int(time.time()))
        a_np = np.random.rand(M).astype(np.float32)
        w_np = np.random.rand(N).astype(np.float32)

        # Time the optimized implementation
        a = tvm.nd.array(a_np, dev)
        w = tvm.nd.array(w_np, dev)
        b = tvm.nd.array(np.zeros((M + N - 1), dtype='float32'), dev)

        func(a, w, b)

    time_tvm = timeit.timeit(tvm_time, number=n_repeat)
    return time_tvm

test_speed_conv1d_cpu()

0.33159597500002747

In [ ]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_conv1d_cpu_scheduler

M = 4096
N = 128
dtype = 'float32'
a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
b_np = np.convolve(a_np, w_np)

s, A, W, B = make_conv1d_cpu_scheduler(M, N)
func = tvm.build(s, [A, W, B], "llvm")

dev = tvm.cpu()
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M+N-1), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat=100)


print("Answer:", b_np)
print("Output:", b)
print(f"1D conv TVM runtime: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [0.26200765 0.830693   1.2422107  ... 0.5778411  0.7431597  0.5678245 ]
Output: [0.26200765 0.830693   1.2422107  ... 0.5778411  0.7431597  0.5678245 ]
1D conv TVM runtime: 32.971833 ms


In [ ]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((4096,), "float32"), W: T.Buffer((128,), "float32"), B: T.Buffer((4223,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "global_symbol": "main", "tir.noalias": T.bool(True)})
        for n_outer in T.parallel(132):
            B_global = T.allocate([1], "float32", "global")
            for n_inner in range(32):
                B_global_1 = T.Buffer((1,), data=B_global, align=4)
                B_global_1[0] = T.float32(0)
                if T.likely(n_outer * 32 + n_inner < 4223):
                    for k_outer, k_inner in T.grid(132, 32):
                        if T.likely(k_outer * 32 + k_inner < 4223):
                            cse_var_2: T.int32 = k_outer * 32
                            cse_var_1: T.int32 = n_outer * 32 + n_inner - k_inner - cse_var_2
                            A_1 = T.Buffer((4096,), data=A.data

In [ ]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_1dconv_cpu.py

/content/gdrive/MyDrive/ece5545/a3-yang-su2000
============================= test session starts ==============================
platform linux -- Python 3.9.16, pytest-7.2.2, pluggy-1.0.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-yang-su2000
plugins: anyio-3.6.2
collected 15 items                                                             

tests/test_1dconv_cpu.py ...............                                 [100%]

============================== 15 passed in 9.70s ==============================
